In [ ]:
import numpy as np
from keras.layers import Input, Convolution2D, Dense, Dropout, Flatten, concatenate, BatchNormalization
from keras.models import Model  # basic class for specifying and training a neural network
from keras import losses
import keras

import tensorflow as tf

In [ ]:
SIZE = 13

In [ ]:
npz = np.load('models/train_vectors_13_13_r2.npz')
train_vectors = npz['train_vectors']
train_p = npz['train_p']
train_q = npz['train_q']

In [ ]:
inp = Input(shape=(SIZE, SIZE, 1))

bn1 = BatchNormalization()(inp)
conv_1 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn1)
bn2 = BatchNormalization()(conv_1)
conv_2 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn2)
bn3 = BatchNormalization()(conv_2)
conv_3 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn3)
bn4 = BatchNormalization()(conv_3)

flat = Flatten()(bn4)
turn_input = Input(shape=(1,), name='turn')
full = concatenate([flat, turn_input])

hidden = Dense(15, activation='relu', kernel_initializer='random_uniform')(full)
bn4 = BatchNormalization()(hidden)

out = Dense(1)(bn4)

q_model = Model(inputs=[inp, turn_input], outputs=out)
q_model.compile(loss=losses.mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])

In [ ]:
boards = np.array([board for board in train_vectors[:, 0]]).reshape(train_vectors.shape[0], SIZE, SIZE, 1)


In [ ]:
with tf.device('/gpu:0'):
    model.fit([boards, 
               train_vectors[:, 1]], 
              train_q, 
              epochs=10,
             validation_split=0.1)

In [ ]:
model.save('models/13_13_q_v2.net')

In [ ]:
inp = Input(shape=(SIZE, SIZE, 1))

bn1 = BatchNormalization()(inp)
conv_1 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn1)
bn2 = BatchNormalization()(conv_1)
conv_2 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn2)
bn3 = BatchNormalization()(conv_2)
conv_3 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn3)
bn4 = BatchNormalization()(conv_3)

flat = Flatten()(bn4)
turn_input = Input(shape=(1,), name='turn')
full = concatenate([flat, turn_input])

hidden = Dense(15, activation='relu', kernel_initializer='random_uniform')(full)
bn4 = BatchNormalization()(hidden)

out = Dense(SIZE ** 2, activation='softmax')(bn4)

p_model = Model(inputs=[inp, turn_input], outputs=out)
p_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    p_model.fit([boards, 
               train_vectors[:, 1]], 
              train_p, 
              epochs=10,
             validation_split=0.1)

In [ ]:
p_model.save('models/13_13_p_v2.net')